# Cài đặt - liên kết tới api của IBM 
**Chú ý** các phiên làm việc chỉ lưu trữ trong 12 tiếng, và tự động xóa sau khi kết thúc phiên, nếu muốn lưu trữ trên drive, hãy chỉnh sửa code.


In [ ]:
credentials = {
  "bucket": "signlook-bucket",
  "access_key_id": "3af70ddf83ff47688873205c0b8801f5",
  "secret_access_key": "db0ab6972b55864d1b57b52f1657139f1830087b4d0cef6e",
  "endpoint_url": "https://s3.us.cloud-object-storage.appdomain.cloud"
}

In [ ]:
NUM_TRAIN_STEPS = 500
MODEL_TYPE = 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18'
CONFIG_TYPE = 'ssd_mobilenet_v1_quantized_300x300_coco14_sync'

import os
CLOUD_ANNOTATIONS_MOUNT = os.path.join('/content', credentials['bucket'])
ANNOTATIONS_JSON_PATH   = os.path.join(CLOUD_ANNOTATIONS_MOUNT, '_annotations.json')

CHECKPOINT_PATH = '/content/checkpoint'
OUTPUT_PATH     = '/content/output'
EXPORTED_PATH   = '/content/exported'
DATA_PATH       = '/content/data'

LABEL_MAP_PATH    = os.path.join(DATA_PATH, 'label_map.pbtxt')
TRAIN_RECORD_PATH = os.path.join(DATA_PATH, 'train.record')
VAL_RECORD_PATH   = os.path.join(DATA_PATH, 'val.record')

# Cài đặt object detection api
Git clone file về cloud.

### Gói phụ thuộc
Sử dụng các gói để vận hành khả năng xử lý của colab trên cloud.

### Protocol Buffers
Dùng để cài đặt tf object api và biên dịch.

### Environment
Khai báo đường dẫn của object api để python có thể nhận diện.

In [ ]:
%tensorflow_version 1.x
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/cloud-annotations/models

!pip install cloud-annotations==0.0.4
!pip install tf_slim
!pip install lvis
!pip install --no-deps tensorflowjs==1.4.0

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

pwd = os.getcwd()
os.environ['PYTHONPATH'] += f':{pwd}:{pwd}/slim'

TensorFlow 1.x selected.
Cloning into 'models'...
remote: Enumerating objects: 2282, done.
remote: Counting objects: 100% (2282/2282), done.
remote: Compressing objects: 100% (1976/1976), done.
remote: Total 2282 (delta 555), reused 947 (delta 280), pack-reused 0
Receiving objects: 100% (2282/2282), 30.55 MiB | 32.83 MiB/s, done.
Resolving deltas: 100% (555/555), done.
     |████████████████████████████████| 358kB 11.7MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
/content/models/research


# Kiểm tra cài đặt


In [ ]:
!python object_detection/builders/model_builder_tf1_test.py

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_model_from_config_wi

# Kết nối Cloud Annotations Bucket
Để dùng dataset từ IBM, cần phải kết nối tới Colab.

In [ ]:
import cloud_annotations as ca
ca.mount(CLOUD_ANNOTATIONS_MOUNT, credentials)

installing s3fs...
done
signlook-bucket mounted


# Tạo label map
Các dữ liệu sẽ được lưu trữ thành các dataset và đánh số theo id để máy có thể nhận diện được.

Bên dưới ví dụ là cấu trúc của dataset:
```
item {
  id: 1
  name: 'Hieu'
}

item {
  id: 2
  name: 'Khong hieu'
}

item {
  id: 3
  name: 'Xin loi'
}
```


In [ ]:
import os
import json

# Get a list of labels from the annotations.json
labels = {}
with open(ANNOTATIONS_JSON_PATH) as f:
  annotations = json.load(f)
  labels = annotations['labels']

# Create a file named label_map.pbtxt
os.makedirs(DATA_PATH, exist_ok=True)
with open(LABEL_MAP_PATH, 'w') as f:
  # Loop through all of the labels and write each label to the file with an id
  for idx, label in enumerate(labels):
    f.write('item {\n')
    f.write("\tname: '{}'\n".format(label))
    f.write('\tid: {}\n'.format(idx + 1)) # indexes must start at 1
    f.write('}\n')

# Tạo TFRecords
Tạo dữ liệu theo format tfrecord cho tf object api có thể sử dụng dataset.
```
{
  'image/height': 1800,
  'image/width': 2400,
  'image/filename': 'image1.jpg',
  'image/source_id': 'image1.jpg',
  'image/encoded': ACTUAL_ENCODED_IMAGE_DATA_AS_BYTES,
  'image/format': 'jpeg',
  'image/object/bbox/xmin': [0.7255949630314233, 0.8845598428835489],
  'image/object/bbox/xmax': [0.9695875693160814, 1.0000000000000000],
  'image/object/bbox/ymin': [0.5820120073891626, 0.1829972290640394],
  'image/object/bbox/ymax': [1.0000000000000000, 0.9662484605911330],
  'image/object/class/text': (['Cat', 'Dog']),
  'image/object/class/label': ([1, 2])
}
```

In [ ]:
import os
import io
import json
import random

import PIL.Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

def create_tf_record(images, annotations, label_map, image_path, output):
  # Create a train.record TFRecord file.
  with tf.python_io.TFRecordWriter(output) as writer:
    # Loop through all the training examples.
    for image_name in images:
      try:
        # Make sure the image is actually a file
        img_path = os.path.join(image_path, image_name)   
        if not os.path.isfile(img_path):
          continue
          
        # Read in the image.
        with tf.gfile.GFile(img_path, 'rb') as fid:
          encoded_jpg = fid.read()

        # Open the image with PIL so we can check that it's a jpeg and get the image
        # dimensions.
        encoded_jpg_io = io.BytesIO(encoded_jpg)
        image = PIL.Image.open(encoded_jpg_io)
        if image.format != 'JPEG':
          raise ValueError('Image format not JPEG')

        width, height = image.size

        # Initialize all the arrays.
        xmins = []
        xmaxs = []
        ymins = []
        ymaxs = []
        classes_text = []
        classes = []

        # For each image, loop through all the annotations and append their values.
        for a in annotations[image_name]:
          if ("x" in a and "x2" in a and "y" in a and "y2" in a):
            label = a['label']
            xmins.append(a["x"])
            xmaxs.append(a["x2"])
            ymins.append(a["y"])
            ymaxs.append(a["y2"])
            classes_text.append(label.encode("utf8"))
            classes.append(label_map[label])
       
        # Create the TFExample.
        tf_example = tf.train.Example(features=tf.train.Features(feature={
          'image/height': dataset_util.int64_feature(height),
          'image/width': dataset_util.int64_feature(width),
          'image/filename': dataset_util.bytes_feature(image_name.encode('utf8')),
          'image/source_id': dataset_util.bytes_feature(image_name.encode('utf8')),
          'image/encoded': dataset_util.bytes_feature(encoded_jpg),
          'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
          'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
          'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
          'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
          'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
          'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
          'image/object/class/label': dataset_util.int64_list_feature(classes),
        }))
        if tf_example:
          # Write the TFExample to the TFRecord.
          writer.write(tf_example.SerializeToString())
      except ValueError:
        print('Invalid example, ignoring.')
        pass
      except IOError:
        print("Can't read example, ignoring.")
        pass

with open(ANNOTATIONS_JSON_PATH) as f:
  annotations = json.load(f)['annotations']
  image_files = [image for image in annotations.keys()]
  # Load the label map we created.
  label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)

  random.seed(42)
  random.shuffle(image_files)
  num_train = int(0.7 * len(image_files))
  train_examples = image_files[:num_train]
  val_examples = image_files[num_train:]

  create_tf_record(train_examples, annotations, label_map, CLOUD_ANNOTATIONS_MOUNT, TRAIN_RECORD_PATH)
  create_tf_record(val_examples, annotations, label_map, CLOUD_ANNOTATIONS_MOUNT, VAL_RECORD_PATH)

# Tải base model

Việc đào tạo một mô hình từ đầu có thể tốn thời gian. Để giảm thiểu điều này bằng cách sử dụng điểm kiểm tra mô hình được đào tạo trước. Thay vì bắt đầu từ con số không, chúng ta có thể thêm vào những gì đã học bằng dữ liệu tùy biến.

Đây là pretrained model checkpoints tải từ [model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md).

In [ ]:
import os
import tarfile

import six.moves.urllib as urllib

download_base = 'http://download.tensorflow.org/models/object_detection/'
model = MODEL_TYPE + '.tar.gz'
tmp = '/content/checkpoint.tar.gz'

if not (os.path.exists(CHECKPOINT_PATH)):
  # Download the checkpoint
  opener = urllib.request.URLopener()
  opener.retrieve(download_base + model, tmp)

  # Extract all the `model.ckpt` files.
  with tarfile.open(tmp) as tar:
    for member in tar.getmembers():
      member.name = os.path.basename(member.name)
      if 'model.ckpt' in member.name:
        tar.extract(member, path=CHECKPOINT_PATH)

  os.remove(tmp)

[link text](https://)# Tùy chỉnh Model

Điều cuối cùng cần làm là đưa vào pipeline: số lượng nhãn và nơi để tìm label map, TFRecord và điểm kiểm tra mô hình. Chúng tôi cũng cần thay đổi kích thước lô batch, vì kích thước batch mặc định là 128 quá lớn để Colab xử lý.

In [ ]:
import re

from google.protobuf import text_format

from object_detection.utils import config_util
from object_detection.utils import label_map_util

pipeline_skeleton = '/content/models/research/object_detection/samples/configs/' + CONFIG_TYPE + '.config'
configs = config_util.get_configs_from_pipeline_file(pipeline_skeleton)

label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)
num_classes = len(label_map.keys())
meta_arch = configs["model"].WhichOneof("model")

override_dict = {
  'model.{}.num_classes'.format(meta_arch): num_classes,
  'train_config.batch_size': 32,
  'train_input_path': TRAIN_RECORD_PATH,
  'eval_input_path': VAL_RECORD_PATH,
  'train_config.fine_tune_checkpoint': os.path.join(CHECKPOINT_PATH, 'model.ckpt'),
  'label_map_path': LABEL_MAP_PATH
}

configs = config_util.merge_external_params_with_configs(configs, kwargs_dict=override_dict)
pipeline_config = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_config, DATA_PATH)

INFO:tensorflow:Maybe overwriting model.ssd.num_classes: 9
INFO:tensorflow:Maybe overwriting train_config.batch_size: 24
INFO:tensorflow:Maybe overwriting train_input_path: /content/data/train.record
INFO:tensorflow:Maybe overwriting eval_input_path: /content/data/val.record
INFO:tensorflow:Maybe overwriting train_config.fine_tune_checkpoint: /content/checkpoint/model.ckpt
INFO:tensorflow:Maybe overwriting label_map_path: /content/data/label_map.pbtxt
INFO:tensorflow:Writing pipeline config file to /content/data/pipeline.config


# Bắt đầu huấn luyện
Chúng ta có thể bắt đầu quá trình đào tạo bằng cách gọi tập lệnh `model_main`, truyền:
- Vị trí của `pipepline.config` đã tạo
- Nơi muốn lưu mô hình
- Số bước muốn huấn luyện model (đào tạo càng lâu thì càng chính xác)
- Số lượng các bước đánh giá (hoặc tần suất kiểm tra mô hình) cho biết mô hình hoạt động tốt như thế nào

In [ ]:
!rm -rf $OUTPUT_PATH
!python -m object_detection.model_main \
    --pipeline_config_path=$DATA_PATH/pipeline.config \
    --model_dir=$OUTPUT_PATH \
    --num_train_steps=$NUM_TRAIN_STEPS \
    --num_eval_steps=500

W1221 14:58:30.910344 140326522623872 model_lib.py:801] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 500
I1221 14:58:30.910540 140326522623872 config_util.py:552] Maybe overwriting train_steps: 500
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1221 14:58:30.910639 140326522623872 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I1221 14:58:30.910722 140326522623872 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1221 14:58:30.910806 140326522623872 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1221 14:58:30.910914 140326522623872 model_lib.py:817] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_tpu

# Xuất inference graph

Sau khi mô hình của bạn đã được đào tạo, bạn có thể có một vài điểm kiểm tra. Một điểm kiểm tra thường được phát ra sau mỗi 500 bước đào tạo. Mỗi điểm kiểm tra là một ảnh chụp nhanh mô hình của bạn tại thời điểm đó trong quá trình đào tạo. Trong trường hợp quá trình đào tạo đang chạy dài bị lỗi, có thể chọn ở điểm kiểm tra cuối cùng thay vì bắt đầu lại từ đầu.



In [ ]:
import os
import re

regex = re.compile(r"model\.ckpt-([0-9]+)\.index")
numbers = [int(regex.search(f).group(1)) for f in os.listdir(OUTPUT_PATH) if regex.search(f)]
TRAINED_CHECKPOINT_PREFIX = os.path.join(OUTPUT_PATH, 'model.ckpt-{}'.format(max(numbers)))

print(f'Using {TRAINED_CHECKPOINT_PREFIX}')

!rm -rf $EXPORTED_PATH
!python -m object_detection.export_inference_graph \
  --pipeline_config_path=$DATA_PATH/pipeline.config \
  --trained_checkpoint_prefix=$TRAINED_CHECKPOINT_PREFIX \
  --output_directory=$EXPORTED_PATH

Using /content/output/model.ckpt-500
Instructions for updating:
Please use `layer.__call__` method instead.
W1221 15:08:06.178596 140091111151488 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I1221 15:08:07.615123 140091111151488 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I1221 15:08:07.652612 140091111151488 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I1221 15:08:07.690379 140091111151488 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:ten

# Chuyển đổi sang tfjs
Để sử dụng model trên web broswer


In [ ]:
!tensorflowjs_converter \
  --input_format=tf_frozen_model \
  --output_format=tfjs_graph_model \
  --output_node_names='Postprocessor/ExpandDims_1,Postprocessor/Slice' \
  --quantization_bytes=1 \
  --skip_op_check \
  $EXPORTED_PATH/frozen_inference_graph.pb \
  /content/model_web

import json

from object_detection.utils.label_map_util import get_label_map_dict

label_map = get_label_map_dict(LABEL_MAP_PATH)
label_array = [k for k in sorted(label_map, key=label_map.get)]

with open(os.path.join('/content/model_web', 'labels.json'), 'w') as f:
  json.dump(label_array, f)

!cd /content/model_web && zip -r /content/model_web.zip *

2020-12-21 15:08:20.992350: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:786] Optimization results for grappler item: graph_to_optimize
2020-12-21 15:08:20.992411: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   debug_stripper: Graph size after: 3565 nodes (0), 4556 edges (0), time = 3.051ms.
2020-12-21 15:08:20.992423: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   model_pruner: Graph size after: 2100 nodes (-1465), 2355 edges (-2201), time = 18.037ms.
2020-12-21 15:08:20.992438: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   constant_folding: Graph size after: 625 nodes (-1475), 687 edges (-1668), time = 237.36ms.
2020-12-21 15:08:20.992451: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   arithmetic_optimizer: Graph size after: 482 nodes (-143), 646 edges (-41), time = 40.307ms.
2020-12-21 15:08:20.992463: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:788]   dependency_optimizer: Graph size after: 476 

# Tải model tfjs

In [ ]:
from google.colab import files
files.download('/content/model_web.zip') 

# Chuyển đổi sang tflite

In [ ]:
%cd /content
!mkdir model_android

/content


In [ ]:
%%bash
python /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --pipeline_config_path /content/data/pipeline.config \
    --trained_checkpoint_dir /content/output/checkpoint \
    --output_directory /content/model_android

Traceback (most recent call last):
  File "/content/models/research/object_detection/export_tflite_graph_tf2.py", line 126, in <module>
    app.run(main)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 300, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "/content/models/research/object_detection/export_tflite_graph_tf2.py", line 122, in main
    FLAGS.ssd_use_regular_nms)
  File "/content/models/research/object_detection/export_tflite_graph_lib_tf2.py", line 231, in export_tflite_model
    pipeline_config.model, is_training=False)
  File "/content/models/research/object_detection/builders/model_builder.py", line 1090, in build
    add_summaries)
  File "/content/models/research/object_detection/builders/model_builder.py", line 380, in _build_ssd_model
    is_training=is_training)
  File "/content/models/research/object_detection/builders/model_builder.py", line 293, i

In [ ]:
!tflite_convert --saved_model_dir=/content/exported/saved_model --output_file= /content/model_android/model.tflite

2020-12-21 15:16:13.456820: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-12-21 15:16:13.492857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-21 15:16:13.493519: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2020-12-21 15:16:13.493813: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-12-21 15:16:13.495684: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-12-21 15:16:13.497414: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2020-

# Tải model tflite 

In [ ]:
from google.colab import files
files.download('/content/model_android')

# Sử dụng model
Dùng `model_web` và `model_android` đã tải xuống, tham khảo template của IBM để chạy thử [mô hình javascript](https://github.com/cloud-annotations/object-detection-react)  hoặc [mô hình Android](https://github.com/cloud-annotations/object-detection-android)

# Tài liệu tham khảo
- https://cloud.annotations.ai/workshops/object-detection/6.html
- https://cloud.annotations.ai/buckets/